In [1]:
import tensorflow as tf

2025-03-25 03:28:14.918755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-25 03:28:14.918900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-25 03:28:15.054639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Convert Images in directory into Dataset

we can use `tf.keras.preprocessing.image_dataset_from_directory` to convert the data into dataset so we can train the models out of the box


In [2]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/kidney-my-rep/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
    shuffle=True,
    batch_size=32,
    image_size=(299, 299),
)

labels = dataset.class_names
labels

Found 12446 files belonging to 4 classes.


['Cyst', 'Normal', 'Stone', 'Tumor']

In [3]:
import numpy as np

for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    break

(32, 299, 299, 3)
[3 1 0 3 0 1 1 0 1 1 0 0 1 0 0 0 0 1 3 3 3 1 0 1 3 3 3 3 1 1 0 1]


In [4]:
# train test split
train_size = int(0.8 * len(dataset))
test_size = int(0.2 * len(dataset))
train_size, test_size

(311, 77)

## Train, Test, Validate

partition the data into train test and validation datasets


In [5]:
def get_dataset_partisions_tf(ds, train_split=0.8, test_split=0.2, shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * len(ds))
    test_size = int(test_split * len(ds))
    train_ds = ds.take(train_size)
    test_ds = ds.skip(train_size)
    val_ds = test_ds.skip(test_size)
    test_ds = test_ds.take(test_size)
    return train_ds, test_ds, val_ds

In [6]:
train_ds, test_ds, val_ds = get_dataset_partisions_tf(dataset)
len(train_ds), len(test_ds), len(val_ds)

(311, 77, 1)

## Resize and Normalize

- Xception models takes the image input as 299x299 pixels so converting into the trainable format is necessary
- The Images are to be normalized before to train accurately and efficiently


In [7]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(299, 299),
    tf.keras.layers.Rescaling(1./255)
])

### Download the Xception model predefined weights from tensorflow into your working environment


In [9]:
# train using Xception
base_model = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False,
    pooling='avg',
    classifier_activation='softmax',
    classes=len(labels)
)

base_model.trainable = False

inputs = tf.keras.Input(shape=(299, 299, 3))
x = resize_and_rescale(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


In [11]:


model.compile(
    optimizer='AdamW',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=32,
    epochs=1
)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 2048)           │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,124,268 (80.58 MB)

 Trainable params: 262,788 (1.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

2025-03-25 03:30:40.101018: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=2} for conv (f32[32,128,147,147]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,147,147]{3,2,1,0}, f32[128,1,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, feature_group_count=128, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2025-03-25 03:30:40.105183: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.004301834s
Trying algorithm eng3{k11=2} for conv (f32[32,128,147,147]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,147,147]{3,2,1,0}, f32[128,1,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, feature_group_count=128, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking 

311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - accuracy: 0.7094 - loss: 0.7480

W0000 00:00:1742873557.525545      94 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


311/311 ━━━━━━━━━━━━━━━━━━━━ 159s 316ms/step - accuracy: 0.7098 - loss: 0.7472 - val_accuracy: 0.9375 - val_loss: 0.1959


In [12]:
model.evaluate(test_ds)

77/77 ━━━━━━━━━━━━━━━━━━━━ 40s 237ms/step - accuracy: 0.9500 - loss: 0.2154


[0.2123449444770813, 0.9476460814476013]

In [14]:
import tensorflow as tf

# Define the base model (Xception)
base_model = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=(299, 299, 3),
    include_top=False,
    pooling='avg'
)

# Initially freeze the entire base model
base_model.trainable = True  # Set to True to allow fine-tuning

# Get the total number of layers in the base model
total_layers = len(base_model.layers)
print(f"Total layers in Xception: {total_layers}")

# Freeze all layers except the last 50
for layer in base_model.layers[:-50]:
    layer.trainable = False



# Define the input
inputs = tf.keras.Input(shape=(299, 299, 3))

# Assuming resize_and_rescale is a preprocessing function you've defined
x = resize_and_rescale(inputs)
x = base_model(x, training=False)  # Use training=False to keep batch norm layers frozen
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(labels), activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)



Total layers in Xception: 133


In [15]:


model.compile(
    optimizer='AdamW',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=32,
    epochs=1
)

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 2048)           │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,124,268 (80.58 MB)

 Trainable params: 12,431,092 (47.42 MB)

 Non-trainable params: 8,693,176 (33.16 MB)

...

278/311 ━━━━━━━━━━━━━━━━━━━━ 14s 432ms/step - accuracy: 0.8851 - loss: 0.3315

W0000 00:00:1742873865.922676      94 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 577ms/step - accuracy: 0.8918 - loss: 0.3132

W0000 00:00:1742873904.877008      95 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


311/311 ━━━━━━━━━━━━━━━━━━━━ 256s 658ms/step - accuracy: 0.8920 - loss: 0.3127 - val_accuracy: 1.0000 - val_loss: 0.0260


In [16]:
model.evaluate(test_ds)

77/77 ━━━━━━━━━━━━━━━━━━━━ 42s 250ms/step - accuracy: 0.9791 - loss: 0.0642


[0.07359074056148529, 0.9772727489471436]

In [19]:
import tensorflow as tf

# Define input shape
input_shape = (299, 299, 3)

# Load Xception base model
xception_base = tf.keras.applications.Xception(
    weights='imagenet',
    input_shape=input_shape,
    include_top=False,
    pooling='avg'
)

# Load ResNet50V2 base model
resnet50v2_base = tf.keras.applications.ResNet50V2(
    weights='imagenet',
    input_shape=input_shape,
    include_top=False,
    pooling='avg'
)

# Freeze both base models initially
xception_base.trainable = False
resnet50v2_base.trainable = False

# Define a single input layer
inputs = tf.keras.Input(shape=input_shape)

# Preprocessing (example resize_and_rescale function)
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(299, 299),
    tf.keras.layers.Rescaling(1./255)
])
x = resize_and_rescale(inputs)

# Pass the input through both models
xception_output = xception_base(x, training=False)
resnet50v2_output = resnet50v2_base(x, training=False)

# Concatenate the outputs
combined = tf.keras.layers.Concatenate()([xception_output, resnet50v2_output])

# Add custom layers
x = tf.keras.layers.Dense(256, activation='relu')(combined)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)  # Assuming 4 classes

# Create the hybrid model
hybrid_model = tf.keras.Model(inputs, outputs)

# Fine-tune: Unfreeze last 50 layers of each model
xception_base.trainable = True
resnet50v2_base.trainable = True
for layer in xception_base.layers[:-50]:
    layer.trainable = False
for layer in resnet50v2_base.layers[:-50]:
    layer.trainable = False

# Compile with sparse_categorical_crossentropy for integer labels
hybrid_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',  # Use this for integer labels
    metrics=['accuracy']
)



In [20]:
history = hybrid_model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size=32,
    epochs=1
)

W0000 00:00:1742874378.194347      92 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


310/311 ━━━━━━━━━━━━━━━━━━━━ 0s 638ms/step - accuracy: 0.8988 - loss: 0.2892

W0000 00:00:1742874615.455118      94 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


311/311 ━━━━━━━━━━━━━━━━━━━━ 0s 765ms/step - accuracy: 0.8991 - loss: 0.2886

W0000 00:00:1742874645.074524      94 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


311/311 ━━━━━━━━━━━━━━━━━━━━ 351s 860ms/step - accuracy: 0.8993 - loss: 0.2880 - val_accuracy: 1.0000 - val_loss: 2.3760e-04


In [21]:
model.evaluate(test_ds)

77/77 ━━━━━━━━━━━━━━━━━━━━ 41s 254ms/step - accuracy: 0.9753 - loss: 0.0882


[0.09530986100435257, 0.9724025726318359]

In [ ]:
# predict with new images
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Cyst/Cyst- (1003).jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch
predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal/Normal- (1006).jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch
predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1005).jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch
predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
import numpy as np

img = tf.keras.preprocessing.image.load_img(
    '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Tumor/Tumor- (1007).jpg', target_size=(299, 299)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch
predictions = model.predict(img_array)
score = tf.nn.sigmoid(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
# plot accuracy and loss
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
model.save('model.keras')